In [1]:
from dotenv import load_dotenv

from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

from direct import root_agent

In [2]:
load_dotenv()

True

In [3]:
session_service = InMemorySessionService()

In [4]:
APP_NAME = "currency"
USER_ID = "user_1"
SESSION_ID = "session_001" # Using a fixed ID for simplicity

In [5]:
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

In [6]:
print(f"Session created: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

Session created: App='currency', User='user_1', Session='session_001'


In [7]:
runner = Runner(
    agent=root_agent, # The agent we want to run
    app_name=APP_NAME,   # Associates runs with our app
    session_service=session_service # Uses our session manager
)

In [8]:
from google.genai import types # For creating message Content/Parts

async def call_agent_async(query: str, runner, user_id, session_id):
  """Sends a query to the agent and prints the final response."""
  print(f"\n>>> User Query: {query}")

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "Agent did not produce a final response." # Default

  # Key Concept: run_async executes the agent logic and yields Events.
  # We iterate through events to find the final answer.
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # You can uncomment the line below to see *all* events during execution
      print(f"  [Event] Author: {event.author}, Type: {type(event).__name__}, Final: {event.is_final_response()}, Content: {event.content}")

      # Key Concept: is_final_response() marks the concluding message for the turn.
      if event.is_final_response():
          if event.content and event.content.parts:
             # Assuming text response in the first part
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # Handle potential errors/escalations
             final_response_text = f"Agent escalated: {event.error_message or 'No specific message.'}"
          # Add more checks here if needed (e.g., specific error codes)
          break # Stop processing events once the final response is found

  print(f"<<< Agent Response: {final_response_text}")


In [9]:
await call_agent_async(
    "How many euro I can get for 100?",
    runner=runner,
    user_id=USER_ID,
    session_id=SESSION_ID
)


>>> User Query: How many euro I can get for 100?
  [Event] Author: currency_agent, Type: Event, Final: True, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='Could you please specify the currency you want to convert from?\n')] role='model'
<<< Agent Response: Could you please specify the currency you want to convert from?



In [10]:
await call_agent_async(
    "100 USD",
    runner=runner,
    user_id=USER_ID,
    session_id=SESSION_ID
)


>>> User Query: 100 USD


  [Event] Author: currency_agent, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-b998c106-d8a5-440a-b8f8-6473f5ba1b62', args={'to_currency': 'EUR', 'from_currency': 'USD'}, name='get_conversion_rate'), function_response=None, text=None)] role='model'
  [Event] Author: currency_agent, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-b998c106-d8a5-440a-b8f8-6473f5ba1b62', name='get_conversion_rate', response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='0.878272', annotations=None)], isError=False)}), text=None)] role='user'


  [Event] Author: currency_agent, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=FunctionCall(id='adk-d525fdf1-1da4-4b04-af52-a0b87b9ad78e', args={'b': 0.878272, 'a': 100}, name='multiplication'), function_response=None, text=None)] role='model'
  [Event] Author: currency_agent, Type: Event, Final: False, Content: parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=FunctionResponse(will_continue=None, scheduling=None, id='adk-d525fdf1-1da4-4b04-af52-a0b87b9ad78e', name='multiplication', response={'result': CallToolResult(meta=None, content=[TextContent(type='text', text='87.8272', annotations=None)], isError=False)}), text=None)] role='user'
  [Event] Author: currency_agent, Type: Event, Fina